# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os, glob
import psycopg2

import pandas as pd

In [2]:
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
type(conn), type(cur)

(psycopg2.extensions.connection, psycopg2.extensions.cursor)

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

## Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('./data/song_data')

In [7]:
path_first_song = song_files[0]

In [8]:
df = pd.read_json(path_first_song, lines= True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# Round the song duration to nearest integer.
df['duration'] = df['duration'].round().astype('int')
# song_id, title, artist_id, year, duration
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 219]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_table_insert

'INSERT INTO songs (song_id, title, artist_id, year, duration) VALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;'

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [13]:
# artist_id, name, location, latitude, longitude
cols = ['id', 'name', 'location', 'latitude', 'longitude']
# Add the prefix.
cols = ['artist_' + col for col in cols]
artist_data = df[cols].values.tolist()[0]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('./data/log_data')

In [16]:
path_first_log = log_files[0]
path_first_log

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [17]:
dflog = pd.read_json(path_first_log, lines= True)
dflog.shape

(388, 18)

In [18]:
dflog.head().T

,0,1,2,3,4
artist,Stephen Lynch,Manowar,Morcheeba,Maroon 5,Train
auth,Logged In,Logged In,Logged In,Logged In,Logged In
firstName,Jayden,Jacob,Jacob,Jacob,Jacob
gender,M,M,M,M,M
itemInSession,0,0,1,2,3
lastName,Bell,Klein,Klein,Klein,Klein
length,182.857,247.562,257.41,231.235,216.764
level,free,paid,paid,paid,paid
location,"Dallas-Fort Worth-Arlington, TX","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL","Tampa-St. Petersburg-Clearwater, FL"
method,PUT,PUT,PUT,PUT,PUT


In [19]:
dflog['page'].value_counts()

NextSong     330
Home          33
Downgrade      6
Login          5
Settings       4
Logout         4
Upgrade        3
Help           2
Error          1
Name: page, dtype: int64

In [20]:
dflog = dflog[dflog['page'] == 'NextSong']
dflog.shape

(330, 18)

In [21]:
dflog.dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
# the timestamp is in milliseconds.
dflog['ts'].head()

0    1543537327796
1    1543540121796
2    1543540368796
3    1543540625796
4    1543540856796
Name: ts, dtype: int64

In [23]:
# Convert unix timestamp (in milliseconds) to datetime.
dflog['time_stamp'] = pd.to_datetime(dflog['ts'], unit= 'ms')
dflog['time_stamp'].head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: time_stamp, dtype: datetime64[ns]

In [24]:
# start_time, hour, day, week, month, year, weekday
dftime = dflog[['time_stamp']].copy()

dftime['hour'] = dftime['time_stamp'].dt.hour
dftime['day'] = dftime['time_stamp'].dt.day
dftime['week'] = dftime['time_stamp'].dt.week
dftime['month'] = dftime['time_stamp'].dt.month
dftime['year'] = dftime['time_stamp'].dt.year
dftime['weekday'] = dftime['time_stamp'].dt.weekday

# # Drop the datatime column
# dftime = dftime.drop('time_stamp', axis= 1)
dftime.head(3)

,time_stamp,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4


In [25]:
dftime.dtypes

time_stamp    datetime64[ns]
hour                   int64
day                    int64
week                   int64
month                  int64
year                   int64
weekday                int64
dtype: object

In [26]:
dftime.values.tolist()[:3]

[[Timestamp('2018-11-30 00:22:07.796000'), 0, 30, 48, 11, 2018, 4],
 [Timestamp('2018-11-30 01:08:41.796000'), 1, 30, 48, 11, 2018, 4],
 [Timestamp('2018-11-30 01:12:48.796000'), 1, 30, 48, 11, 2018, 4]]

In [27]:
for log in dftime.values.tolist()[:3]:
    print(log)

[Timestamp('2018-11-30 00:22:07.796000'), 0, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:08:41.796000'), 1, 30, 48, 11, 2018, 4]
[Timestamp('2018-11-30 01:12:48.796000'), 1, 30, 48, 11, 2018, 4]


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
time_table_insert

'INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;'

In [29]:
# start_time, hour, day, week, month, year, weekday
time_data = dftime.values.tolist()
time_data[:2]

[[Timestamp('2018-11-30 00:22:07.796000'), 0, 30, 48, 11, 2018, 4],
 [Timestamp('2018-11-30 01:08:41.796000'), 1, 30, 48, 11, 2018, 4]]

In [30]:
for log in time_data:
    cur.execute(time_table_insert, log)
    conn.commit()

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
dflog.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'time_stamp'],
      dtype='object')

In [32]:
# user_id, first_name, last_name, gender, level
df_user = dflog[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()

In [33]:
df_user.loc[df_user['userId'] == '']

,userId,firstName,lastName,gender,level


In [34]:
df_user.loc[df_user['userId'].isna()]

,userId,firstName,lastName,gender,level


In [35]:
# Remove the rows that `userId` is empty string or NaN.
df_user['userId'] = df_user[df_user['userId'] != '']
df_user = df_user.loc[df_user['userId'].notna()]
# Convert userId to integer.
df_user['userId'] = df_user['userId'].astype('int')
df_user.dtypes

userId        int64
firstName    object
lastName     object
gender       object
level        object
dtype: object

In [36]:
for i, row in df_user.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## 5: songplays Table
### Extract Data and Songplays Table   

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

- Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

Insert Records into Songplays Table

- Implement the songplay_table_insert query and run the cell below to insert records for the songplay actions in this log file into the songplays table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songplays table in the sparkify database.



In [37]:
row1 = dflog.iloc[0]

row1.song, row1.artist, row1.length

("Jim Henson's Dead", 'Stephen Lynch', 182.85668999999999)

In [38]:
dflog.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'time_stamp'],
      dtype='object')

In [39]:
for index, row in dflog.iterrows():

    # Get song_id and artist_id from song and artist tables.
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # Insert songplay record; following values required:
    # start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
    # Note: songplay_id is the primary key and will be filled incrementally.
    ##That's why its datatype is `serial`. 
    songplay_data = (row['time_stamp'], row['userId'], row['level'],
                     song_id, artist_id,
                     row['sessionId'], row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

## Close Connection to Sparkify Database

In [40]:
conn.close()